# Get Started with ReSEn

Welcome to the Reusable Software Environment (ReSEn) platform!  In this tutorial, we will introduce you to many of the packages installed in ReSEn that are specifically useful for geospace science.  We are only going go go over basic use cases here - for detailed explination of packages, please see the tutorial/documentation specific to that package.

## Space science software
Resen gives you the ability to use common comunity packages without installing them.

spacepy allows you to ...

pyglow gives you access to common models

## Python APIs for accessing data
Many python packages come with the ability to fetch and read data files from particular instrument providers.
i.e. mangopy

i. e. mangopy data fetching

Also: MadrigalWeb

## Accessing your own data
You can upload your own data sets to your jupyter notebook.

Try this by downloading our test data set and uploading it to your jupyter server

## Get Results
You can use this dataset with tools available in Resen to quickly write analysis code and visualizing your results

# apexpy
apexpy is a python wrapper around the FORTRAN implimentation of the Apex magnetic coordinate system.  It allows for converting between geodetic, apex, and quasidipole magnetic coordinates, as described by Laundal and Richmond, 2016.

In [10]:
from apexpy import Apex

# Initialize an instance of the Apex class with the year
A = Apex(date=2019.1)

# convert geodetic coordinates to modified apex coordinates
mlat, mlon = A.convert(60,75,'geo','apex',height=300)

# convert modified apex coordinates to geodetic coordinates
glat, glon = A.convert(65,100,'apex','geo',height=300)

# find the magnetic local time of a particular magnetic longitude at a given UT
import datetime as dt
mlt = A.mlon2mlt(120, dt.datetime(2019, 2, 10, 18, 0, 0))

ModuleNotFoundError: No module named 'apexpy'

# spacepy
spacepy is a python package that contains many utilities useful for working with space-based data sources.

In [1]:
import spacepy as space

# time conversion
# coordinate conversion
# omni
# pyCDF?

ModuleNotFoundError: No module named 'spacepy'